In [1]:
#PYTHON
import h5py
import numpy as np
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyspark

#SPARK
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import *
from operator import add

import sys

# Append the path to Pydoop to sys.path
#sys.path.append("/usr/local/lib/python3.8/dist-packages")

Spark Configuration

In [2]:
# Spark Session - 2 CORE - imporvement CONFIGURATION!!
# spark_session = SparkSession.builder\
#     .master("local[2]")\
#     .appName("pseudo_spark_nora")\
#     .getOrCreate()

# Set Hadoop configuration directory
#os.environ['HADOOP_CONF_DIR'] = '/path/to/hadoop/conf'
#spark_session.stop()
#os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

spark_session = SparkSession.builder \
    .appName("HDFS_Connection_Test_nora") \
    .master("yarn") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

# num executors 
# executor memory RAM 


# spark_session = SparkSession.builder \
#     .appName("HDFS_Connection_Test_nora") \
#     .master("yarn") \
#     .config("spark.executor.instances", "2") \
#     .config("spark.hadoop.fs.default.name", "hdfs://master-node:9000") \
#     .config("spark.hadoop.fs.defaultFS", "hdfs://master-node:9000") \
#     .config("spark.hadoop.fs.hdfs.impl", "org.apache.hadoop.hdfs.DistributedFileSystem") \
#     .config("spark.hadoop.dfs.server.namenode.class", "org.apache.hadoop.hdfs.server.namenode.NameNode") \
#     .config("spark.hadoop.conf", "org.apache.hadoop.hdfs.HdfsConfiguration") \
#     .getOrCreate()

#.config("spark.dynamicAllocation.enabled", False)\
#.config("spark.cores.max", 4)\


#.config("spark.jars.packages", "LLNL:spark-hdf5:0.0.4") \
# RDD API
spark_context = spark_session.sparkContext
spark_context.setLogLevel("INFO")

sqlContext = SQLContext(spark_session.sparkContext)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 19:31:33 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
/home/hadoop/.local/lib/python3.8/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


File Path

In [3]:
#file_roots = ['hdfs://130.238.29.183:9000//input/millionsongsubset.tar.gz']
directory_path = "file://data/B/A/A"
#directory_path = "data_AZ"
#Path300
#directory_path = "/user/hadoop/MillionSongSubset/A/A/"
#directory_path = "MillionSongSubset/A/A"
directory_path = "hdfs://master-node:9000/user/hadoop/MillionSongSubset_ASCI_analysis_songs"

In [6]:
df = spark_session.read.json(directory_path)

24/03/14 19:39:05 INFO InMemoryFileIndex: It took 12 ms to list leaf files for 1 paths.
24/03/14 19:39:05 INFO InMemoryFileIndex: It took 16 ms to list leaf files for 11 paths.
24/03/14 19:39:05 INFO FileSourceStrategy: Pushed Filters: 
24/03/14 19:39:05 INFO FileSourceStrategy: Post-Scan Filters: 
24/03/14 19:39:05 INFO MemoryStore: Block broadcast_7 stored as values in memory (estimated size 353.5 KiB, free 364.7 MiB)
24/03/14 19:39:05 INFO MemoryStore: Block broadcast_7_piece0 stored as bytes in memory (estimated size 35.7 KiB, free 364.7 MiB)
24/03/14 19:39:05 INFO BlockManagerInfo: Added broadcast_7_piece0 in memory on master-node:33151 (size: 35.7 KiB, free: 366.1 MiB)
24/03/14 19:39:05 INFO SparkContext: Created broadcast 7 from json at NativeMethodAccessorImpl.java:0
24/03/14 19:39:05 INFO FileSourceScanExec: Planning scan with bin packing, max size: 23071304 bytes, open cost is considered as scanning 4194304 bytes.
24/03/14 19:39:05 INFO SparkContext: Starting job: json at Nat

In [7]:
df.createOrReplaceTempView("my_table")

In [8]:
sqlContext.sql("SELECT * FROM my_table").show()

24/03/14 19:39:34 INFO FileSourceStrategy: Pushed Filters: 
24/03/14 19:39:34 INFO FileSourceStrategy: Post-Scan Filters: 
24/03/14 19:39:34 INFO MemoryStore: Block broadcast_9 stored as values in memory (estimated size 353.4 KiB, free 364.3 MiB)
24/03/14 19:39:34 INFO MemoryStore: Block broadcast_9_piece0 stored as bytes in memory (estimated size 35.5 KiB, free 364.3 MiB)
24/03/14 19:39:34 INFO BlockManagerInfo: Added broadcast_9_piece0 in memory on master-node:33151 (size: 35.5 KiB, free: 366.1 MiB)
24/03/14 19:39:34 INFO SparkContext: Created broadcast 9 from showString at NativeMethodAccessorImpl.java:0
24/03/14 19:39:34 INFO BlockManagerInfo: Removed broadcast_7_piece0 on master-node:33151 in memory (size: 35.7 KiB, free: 366.1 MiB)
24/03/14 19:39:34 INFO BlockManagerInfo: Removed broadcast_7_piece0 on worker-node-2:34115 in memory (size: 35.7 KiB, free: 366.2 MiB)
24/03/14 19:39:34 INFO BlockManagerInfo: Removed broadcast_7_piece0 on worker-node-1:44403 in memory (size: 35.7 KiB,

AnalysisException: Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the
referenced columns only include the internal corrupt record column
(named _corrupt_record by default). For example:
spark.read.schema(schema).csv(file).filter($"_corrupt_record".isNotNull).count()
and spark.read.schema(schema).csv(file).select("_corrupt_record").show().
Instead, you can cache or save the parsed results and then send the same query.
For example, val df = spark.read.schema(schema).csv(file).cache() and then
df.filter($"_corrupt_record".isNotNull).count().

In [9]:
# read the csv from hdfs
df = sqlContext.read.csv(directory_path, header=False, delimiter=,, inferSchema=True).cache()
#df = sqlContext.read.csv("hdfs://master-node:9000/user/hadoop/MillionSongSubset_ASCI/", header=True, inferSchema=True).cache()

TypeError: csv() got an unexpected keyword argument 'delimiter'

In [11]:
rdd = spark_context.textFile(directory_path)

24/03/14 19:42:33 INFO MemoryStore: Block broadcast_10 stored as values in memory (estimated size 358.7 KiB, free 365.2 MiB)
24/03/14 19:42:33 INFO MemoryStore: Block broadcast_10_piece0 stored as bytes in memory (estimated size 33.1 KiB, free 365.2 MiB)
24/03/14 19:42:33 INFO BlockManagerInfo: Added broadcast_10_piece0 in memory on master-node:33151 (size: 33.1 KiB, free: 366.2 MiB)
24/03/14 19:42:33 INFO SparkContext: Created broadcast 10 from textFile at NativeMethodAccessorImpl.java:0


In [18]:
rdd = rdd.map(lambda line: line.strip('{}').split(','))

In [20]:
# Convert each element of the RDD to a string
rdd = rdd.map(lambda row: [str(elem) for elem in row])

In [21]:
# Collect a sample of the RDD and print it
sample = rdd.take(5)  # Take 5 records as a sample
for row in sample:
    print(row)

24/03/14 19:47:21 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/03/14 19:47:21 INFO DAGScheduler: Got job 9 (runJob at PythonRDD.scala:181) with 1 output partitions
24/03/14 19:47:21 INFO DAGScheduler: Final stage: ResultStage 9 (runJob at PythonRDD.scala:181)
24/03/14 19:47:21 INFO DAGScheduler: Parents of final stage: List()
24/03/14 19:47:21 INFO DAGScheduler: Missing parents: List()
24/03/14 19:47:21 INFO DAGScheduler: Submitting ResultStage 9 (PythonRDD[34] at RDD at PythonRDD.scala:53), which has no missing parents
24/03/14 19:47:21 INFO MemoryStore: Block broadcast_15 stored as values in memory (estimated size 9.3 KiB, free 365.1 MiB)
24/03/14 19:47:21 INFO MemoryStore: Block broadcast_15_piece0 stored as bytes in memory (estimated size 5.5 KiB, free 365.1 MiB)
24/03/14 19:47:21 INFO BlockManagerInfo: Added broadcast_15_piece0 in memory on master-node:33151 (size: 5.5 KiB, free: 366.2 MiB)
24/03/14 19:47:21 INFO SparkContext: Created broadcast 15 from broadca

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 4 times, most recent failure: Lost task 0.3 in stage 9.0 (TID 35) (worker-node-2 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/hadoop-hadoop/nm-local-dir/usercache/hadoop/appcache/application_1710430208522_0007/container_1710430208522_0007_01_000002/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/tmp/hadoop-hadoop/nm-local-dir/usercache/hadoop/appcache/application_1710430208522_0007/container_1710430208522_0007_01_000002/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/tmp/hadoop-hadoop/nm-local-dir/usercache/hadoop/appcache/application_1710430208522_0007/container_1710430208522_0007_01_000002/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream


In [13]:
df = rdd.toDF()

24/03/14 19:42:57 INFO FileInputFormat: Total input files to process : 11
24/03/14 19:42:57 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/03/14 19:42:57 INFO DAGScheduler: Got job 5 (runJob at PythonRDD.scala:181) with 1 output partitions
24/03/14 19:42:57 INFO DAGScheduler: Final stage: ResultStage 5 (runJob at PythonRDD.scala:181)
24/03/14 19:42:57 INFO DAGScheduler: Parents of final stage: List()
24/03/14 19:42:57 INFO DAGScheduler: Missing parents: List()
24/03/14 19:42:57 INFO DAGScheduler: Submitting ResultStage 5 (PythonRDD[25] at RDD at PythonRDD.scala:53), which has no missing parents
24/03/14 19:42:57 INFO MemoryStore: Block broadcast_11 stored as values in memory (estimated size 9.0 KiB, free 365.1 MiB)
24/03/14 19:42:57 INFO MemoryStore: Block broadcast_11_piece0 stored as bytes in memory (estimated size 5.5 KiB, free 365.1 MiB)
24/03/14 19:42:57 INFO BlockManagerInfo: Added broadcast_11_piece0 in memory on master-node:33151 (size: 5.5 KiB, free: 366.2 M

In [16]:
df.createOrReplaceTempView("my_table")

In [17]:
sqlContext.sql("SELECT * FROM my_table").show()

24/03/14 19:45:19 INFO DAGScheduler: Got job 7 (showString at NativeMethodAccessorImpl.java:0) with 11 output partitions
24/03/14 19:45:19 INFO DAGScheduler: Final stage: ResultStage 7 (showString at NativeMethodAccessorImpl.java:0)
24/03/14 19:45:19 INFO DAGScheduler: Parents of final stage: List()
24/03/14 19:45:19 INFO DAGScheduler: Missing parents: List()
24/03/14 19:45:19 INFO DAGScheduler: Submitting ResultStage 7 (*(1) Scan ExistingRDD[_1#67]
 MapPartitionsRDD[32] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/03/14 19:45:19 INFO MemoryStore: Block broadcast_13 stored as values in memory (estimated size 18.6 KiB, free 365.1 MiB)
24/03/14 19:45:19 INFO MemoryStore: Block broadcast_13_piece0 stored as bytes in memory (estimated size 9.8 KiB, free 365.1 MiB)
24/03/14 19:45:19 INFO BlockManagerInfo: Added broadcast_13_piece0 in memory on master-node:33151 (size: 9.8 KiB, free: 366.2 MiB)
24/03/14 19:45:19 INFO SparkContext: Created broadcast 13 fr

Py4JJavaError: An error occurred while calling o98.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 7.0 failed 4 times, most recent failure: Lost task 1.3 in stage 7.0 (TID 25) (worker-node-2 executor 1): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 1 fields are required while 2 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$$nestedInanonfun$makeFromJava$16$1.applyOrElse(EvaluatePython.scala:188)
	at org.apache.spark.sql.execution.python.EvaluatePython$.nullSafeConvert(EvaluatePython.scala:213)
	at org.apache.spark.sql.execution.python.EvaluatePython$.$anonfun$makeFromJava$16(EvaluatePython.scala:182)
	at org.apache.spark.sql.SparkSession.$anonfun$applySchemaToPythonRDD$2(SparkSession.scala:875)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForC

In [14]:
df.cache()

DataFrame[_1: string]

In [15]:
df.show()

24/03/14 19:43:20 INFO CodeGenerator: Code generated in 15.614654 ms
24/03/14 19:43:20 INFO DAGScheduler: Got job 6 (showString at NativeMethodAccessorImpl.java:0) with 11 output partitions
24/03/14 19:43:20 INFO DAGScheduler: Final stage: ResultStage 6 (showString at NativeMethodAccessorImpl.java:0)
24/03/14 19:43:20 INFO DAGScheduler: Parents of final stage: List()
24/03/14 19:43:20 INFO DAGScheduler: Missing parents: List()
24/03/14 19:43:20 INFO DAGScheduler: Submitting ResultStage 6 (*(1) Scan ExistingRDD[_1#67]
 MapPartitionsRDD[32] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/03/14 19:43:20 INFO MemoryStore: Block broadcast_12 stored as values in memory (estimated size 18.6 KiB, free 365.1 MiB)
24/03/14 19:43:20 INFO MemoryStore: Block broadcast_12_piece0 stored as bytes in memory (estimated size 9.8 KiB, free 365.1 MiB)
24/03/14 19:43:20 INFO BlockManagerInfo: Added broadcast_12_piece0 in memory on master-node:33151 (size: 9.8 KiB, free: 36

Py4JJavaError: An error occurred while calling o88.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 6.0 failed 4 times, most recent failure: Lost task 1.3 in stage 6.0 (TID 15) (worker-node-2 executor 1): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 1 fields are required while 2 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$$nestedInanonfun$makeFromJava$16$1.applyOrElse(EvaluatePython.scala:188)
	at org.apache.spark.sql.execution.python.EvaluatePython$.nullSafeConvert(EvaluatePython.scala:213)
	at org.apache.spark.sql.execution.python.EvaluatePython$.$anonfun$makeFromJava$16(EvaluatePython.scala:182)
	at org.apache.spark.sql.SparkSession.$anonfun$applySchemaToPythonRDD$2(SparkSession.scala:875)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForC

24/03/14 19:43:23 WARN TaskSetManager: Lost task 0.0 in stage 6.0 (TID 9) (worker-node-1 executor 2): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 1 in stage 6.0 failed 4 times, most recent failure: Lost task 1.3 in stage 6.0 (TID 15) (worker-node-2 executor 1): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 1 fields are required while 2 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$$nestedInanonfun$makeFromJava$16$1.applyOrElse(EvaluatePython.scala:188)
	at org.apache.spark.sql.execution.python.EvaluatePython$.nullSafeConvert(EvaluatePython.scala:213)
	at org.apache.spark.sql.execution.python.EvaluatePython$.$anonfun$makeFromJava$16(EvaluatePython.scala:182)
	at org.apache.spark.sql.SparkSession.$anonfun$applySchemaToPythonRDD$2(SparkSession.scala:875)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.catalyst.expressions.Gen

In [ ]:
df.printSchema()

In [ ]:
df = pd.read_csv(directory_path + 'TRAAAAW128F429D538.asci', delimiter='\t')

In [ ]:
# # Use Spark to list files in HDFS directory
# hdfs_files = spark_session._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration()) \
#     .listStatus(spark_session._jvm.org.apache.hadoop.fs.Path(directory_path))

# 1) Saving the paths to the HDF5 files

In [ ]:
def get_h5_file_paths_hdfs(spark, hdfs_directory_path):
    h5_file_paths = []

    # Use Spark to list files in HDFS directory
    hdfs_files = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration()) \
        .listStatus(spark._jvm.org.apache.hadoop.fs.Path(hdfs_directory_path))

    # Extract file paths from HDFS file status
    for hdfs_file_status in hdfs_files:
        file_path = hdfs_file_status.getPath().toString()

        if hdfs_file_status.isDirectory():
            # If it's a directory, recursively gather paths
            h5_file_paths.extend(get_h5_file_paths_hdfs(spark, file_path))
        elif file_path.endswith(".h5"):
            # If it's an HDF5 file, add to the list
            h5_file_paths.append(file_path)

    return h5_file_paths

In [ ]:
h5_file_paths = get_h5_file_paths_hdfs(spark_session, directory_path)

print('{} HDF5 file paths gathered (HDFS)'.format(len(h5_file_paths)))
for file_path in h5_file_paths[:50]:
    print(file_path)

# 2) List h5_file_paths is converted to RDD

In [ ]:
h5_file_paths[:30]

In [ ]:
file_paths = spark_context.parallelize(h5_file_paths)
file_paths

In [ ]:
file_first = file_paths.first()
file_first

# 3) H5Py file object file creation from the H5 file 

# Debug: Only one h5File

In [ ]:
h5_file_paths = ['hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5']
h5_file_path = 'hdfs://master-node:9000/user/hadoop/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5'

# Version 1

In [ ]:
def extract_info(file_path):
    with h5py.File(file_path, 'r') as file:
        song = file['analysis']['songs']
        np_song = np.array(song[0])
        selected_features = np_song[['danceability', 'duration', 'tempo', 'energy', 'loudness']].ravel()
        return selected_features.tolist()
        #print(f"File found: {file_path}")
         #return file_path
    # except FileNotFoundError:
    #     print(f"File not found: {file_path}")
    #     return None

In [ ]:
file_paths = spark_context.parallelize(h5_file_path)
files_infos_rdd = file_paths.map(extract_info)

In [ ]:
print(files_infos_rdd.collect())

# 4) Create Spark Data Frame

In [ ]:
# Definiere das Schema für den DataFrame
schema = StructType([
    StructField("danceability", FloatType(), True),
    StructField("duration", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("loudness", FloatType(), True),
])

In [ ]:
spark_df = sqlContext.createDataFrame(files_infos_rdd.flatMap(lambda x: x), schema=schema)

In [ ]:
# Zeige den DataFrame-Inhalt
spark_df.show()

# Calculation (Average)

In [ ]:
average_values = spark_df.agg({'danceability': 'avg', 'duration': 'avg', 'tempo': 'avg', 'energy': 'avg', 'loudness': 'avg'}).collect()[0]
print("Average Values::")
for col_name, avg_value in average_values.asDict().items():
    print(f"{col_name}: {avg_value}")

# STOP SPARK

In [ ]:
# release the cores
spark_context.stop()